<a href="https://colab.research.google.com/github/AP-047/RClass-Classification-by-Rational-Approximation/blob/main/notebooks/milestone_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**1. Importing the MNIST dataset and selecting a subset**

In [19]:
import numpy as np
from keras.datasets import mnist

# Load the MNIST dataset
(X_full, y_full), (_, _) = mnist.load_data()

# Initialize containers for subsets
subset_X = []
subset_y = []

# Draw 100 images per digit (0-9)
for digit in range(10):
    digit_indices = np.where(y_full == digit)[0]
    selected_indices = digit_indices[:100]  # Select the first 100 images for this digit
    subset_X.append(X_full[selected_indices])
    subset_y.append(y_full[selected_indices])

# Combine the subsets
subset_X = np.vstack(subset_X)  # Stack all selected images
subset_y = np.hstack(subset_y)  # Stack all selected labels

# Normalize pixel values to [0, 1]
subset_X = subset_X.astype("float32") / 255.0

#____Testing____
print(f"Subset X shape (images): {subset_X.shape}")  # Expected shape: (1000, 28, 28)
print(f"Subset y shape (labels): {subset_y.shape}")  # Expected shape: (1000,)

Subset X shape (images): (1000, 28, 28)
Subset y shape (labels): (1000,)


**2. Dimensionality reduction using PCA**

In [46]:
from sklearn.decomposition import PCA

# Flatten the images to shape (1000, 784)
subset_X_flattened = subset_X.reshape(subset_X.shape[0], -1)

# Apply PCA to reduce to 50 components
pca = PCA(n_components=50)
subset_X_reduced = pca.fit_transform(subset_X_flattened)


#____Testing____
# Print the new shape of the dataset
print(f"Reduced X shape (after PCA): {subset_X_reduced.shape}")
# Print the explained variance ratio to verify the retained variance
explained_variance = np.sum(pca.explained_variance_ratio_)
print(f"Total explained variance retained: {explained_variance * 100:.2f}%")

#____Testing____
# Set the desired label (digit) to inspect
desired_label = 3
# Find the index of the first occurrence of the desired label
index = np.where(subset_y == desired_label)[0][0]
# Get the corresponding reduced vector from PCA
selected_vector = subset_X_reduced[index]
# Print the vector and its length
print(f"Vector for digit {desired_label}: {selected_vector}")
print(f"Length of the vector: {len(selected_vector)}")

Reduced X shape (after PCA): (1000, 50)
Total explained variance retained: 84.14%
Vector for digit 3: [ 2.17378259e+00  2.27192903e+00  5.60351968e-01  3.62105489e+00
 -1.00301944e-01  3.54436684e+00  1.42353439e+00  6.69777095e-01
 -9.05319676e-03  5.96990466e-01  1.07467473e-01  1.01039626e-01
 -7.79237509e-01 -6.47687241e-02  4.35086459e-01 -4.70628738e-02
 -1.33318758e+00  1.05582678e+00 -9.92186666e-02 -3.55662256e-01
 -1.46662667e-01  4.77433264e-01  2.06508845e-01 -1.34056509e+00
 -1.96838498e-01  6.70858085e-01  1.25448406e-01 -3.05725634e-01
 -6.78530633e-01 -1.37480676e-01  2.02044547e-01  1.97795823e-01
 -3.41914967e-02 -1.39779508e-01  4.61459041e-01  6.18021548e-01
  8.46560448e-02  2.88089782e-01  1.42768025e-03  4.39938545e-01
  6.20615005e-01  5.95204979e-02  1.78914055e-01 -7.14286491e-02
  7.75694773e-02 -5.21088019e-03  1.88026175e-01  3.14972103e-02
 -1.08452193e-01 -1.82145447e-01]
Length of the vector: 50


**3. Implement simple multi-variate rational function**

In [47]:
import numpy as np

# Define number of variables and the polynomial degree
num_variables = 50
degree = 4

# Randomly initialize coefficients
np.random.seed(47)  # for reproducibility
nm_cf = np.random.rand(num_variables + 1)  # Numerator coefficients
dn_cf = np.random.rand(num_variables + 1)  # Denominator coefficients

# Define the rational function
def rational_function(x, nm_cf, dn_cf):
    # Construct the numerator (P) and denominator (Q) polynomials
    P = sum(nm_cf[i] * x[i]**degree for i in range(len(x))) + nm_cf[-1]
    Q = sum(dn_cf[i] * x[i]**degree for i in range(len(x))) + dn_cf[-1]

    # Avoid division by zero
    if Q == 0:
        return float('inf')
    return P / Q

#____Testing____
sample_vector = subset_X_reduced[0]  # First PCA-reduced vector
output = rational_function(sample_vector, nm_cf, dn_cf)

print(f"Numerator coefficients (nm_cf): {nm_cf}")
print(f"Denominator coefficients (dn_cf): {dn_cf}")
print(f"Output of the rational function: {output}")

Numerator coefficients (nm_cf): [0.11348847 0.97448309 0.72873463 0.35146781 0.70760514 0.7996046
 0.64556185 0.41459961 0.70603101 0.24664938 0.25599243 0.02401135
 0.09872595 0.30043644 0.64085568 0.32220795 0.18549414 0.91719355
 0.2709208  0.27354789 0.95441268 0.12711457 0.74726485 0.00523796
 0.85679061 0.6959562  0.5530257  0.9352358  0.51262353 0.17761207
 0.53686579 0.29345982 0.01060205 0.88380072 0.65641065 0.94225597
 0.7449453  0.26721045 0.36186288 0.52640389 0.54688497 0.25867032
 0.17463635 0.36071026 0.14018227 0.38907866 0.47188833 0.96882692
 0.14556745 0.51425472 0.52765183]
Denominator coefficients (dn_cf): [0.30524252 0.15967476 0.59691153 0.10372881 0.56796609 0.38630638
 0.08472205 0.56199918 0.64807168 0.66193395 0.18924281 0.95410895
 0.05902446 0.88020624 0.78375816 0.25247741 0.92709052 0.4447506
 0.37715133 0.89382348 0.75405387 0.77445802 0.87904851 0.48148232
 0.30290073 0.44292787 0.52813285 0.60814462 0.52524497 0.97796815
 0.60203988 0.83529596 0.08537

**Optimization Phase**